# ToDo
- [x] get representatives in data
- [x] ideam is the same as previous entry
- [x] get prot seqs from UniProt (API call)
- [x] clean up Pevious FASTA file
- [x] combine results with previous FASTA
- [x] check for duplicate sequences
- [x] extract UniProtID from old CSV and put in new column
- [ ] check for duplicates between both dsets
- [ ] merge CSV files
- [ ] create feature file for protspace3D
- [ ] show results in ProtSpace3D (separate those by others)
- [ ] Split by this group
- [ ] highlight representative
- [ ] only look at french data separatelly

In [1]:
import jupyter_black

jupyter_black.load()

In [1]:
%load_ext autoreload
%autoreload 2

## Prepare data

In [428]:
import importlib
from pathlib import Path
import pandas as pd

import dset_3FTx
import uniprot_helper

# importlib.reload(dset_3FTx)
# importlib.reload(uniprot_helper)

base = Path(".")
fasta_in = base / "raw" / "3and6_new.fasta"  # "3and6_current_alignment.fasta"
csv_in = base / "raw" / "Ivan_3FTx.csv"  # "SM_table.csv"
fasta_out = base / "3FTx.fasta"
csv_out = base / "3FTx.csv"
taxon_mapper_file = Path("raw/taxon_mapper.json")
blast_dir = Path("blast_out")

df = dset_3FTx.construct_df(csv_path=csv_in, fasta_path=fasta_in)
df = dset_3FTx.add_taxon_id(df=df, taxon_mapper_file=taxon_mapper_file)
df = dset_3FTx.get_uniprot_acc_ids(df=df)
df = dset_3FTx.run_blast(df=df, blast_dir=blast_dir)
df = dset_3FTx.manual_curation(df=df)
df = df.dropna(subset="db")
dset_3FTx.save_data(df=df, csv_file=csv_out, fasta_file=fasta_out)
df.head(2)

- found UIDs: 626
- unknown UIDs: 331


BLAST: 100%|██████████| 957/957 [00:00<00:00, 90549.05it/s]


,fasta_id,cystein_group,evolutionary_order,major_group,major_taxon,name,species,seq,taxon_id,acc_id,db,data_origin
0,Cbivi_3FTx_000,Basal,3.0,3FTx,Elapidae,-,Calliophis bivirgatus,LLCRKCNRTVCDLNANCAAGENQCYIVQNKNDTTGRSAIQGCTATC...,8633,A0A898IJN7,TR,NaN
2,Boiga_nigriceps_GGUF01000001,Weird,6.0,3FTx,Colubridae,-,Boiga nigriceps,IICHSCTGRLCTTFQNCPDAQACSQMWKDSDLLKLNIVKGCATNCT...,1867089,A0A6G1S2N1,TR,NaN


## Statistics

In [368]:
def copy_df(df):
    df.index = df.index.fillna("NA")
    pd.io.clipboards.to_clipboard(df.to_markdown(), excel=False)
    print(df)

In [374]:
# Entries found in UniProt
res = df["db"].value_counts(dropna=False)
copy_df(df=res)

SP    462
TR    318
NA    174
Name: db, dtype: int64


In [372]:
# number of UniProt accession IDs with a 100% sequence match
res = df["acc_id"].str.split(",").str.len().value_counts(dropna=False)
copy_df(df=res)

1.0    741
NA     173
2.0     29
3.0      9
5.0      2
Name: acc_id, dtype: int64


In [373]:
# entries that come from genomic that found a 100% match in UniProt
res = df.loc[df["data_origin"] == "genomic", "db"].value_counts(dropna=False)
copy_df(df=res)

TR    78
NA    61
SP    13
Name: db, dtype: int64


## Get UniProt sequence

In [424]:
data = []
for idx, row in df.iterrows():
    if row["acc_id"] is None:
        continue
    acc_ids = row["acc_id"].split(",")
    for acc_id in acc_ids:
        row = row.copy()
        response = uniprot_helper.get_uniprot_entry(acc_id=acc_id, format_type="json")
        response = response.json()
        row["acc_id"] = acc_id
        row["seq"] = response["sequence"]["value"]
        row["prot_evi"] = response["proteinExistence"]
        row["annot_score"] = response["annotationScore"]
        data.append(row)

In [447]:
new_df = pd.DataFrame(data)

col_data = new_df.pop("acc_id")
new_df.insert(loc=0, column="acc_id", value=col_data)

csv_file = Path("../3FTx_full/3FTx.csv")
fasta_file = Path("../3FTx_full/3FTx.fasta")
new_df.to_csv(csv_file, index=False)
with open(fasta_file, "w") as fasta_handler:
    for _, row in new_df.iterrows():
        fasta_handler.write(f">{row['acc_id']}\n")
        fasta_handler.write(f"{row['seq']}\n")


import h5py
esm = "../3FTx_full/3FTx_esm2.h5"
esm_h5 = "../3FTx_full/3FTx_esm2_.h5"
csv_ = "../3FTx_full/3FTx_.csv"
id_tracker = dict()
with h5py.File(esm, "r") as hdf, h5py.File(esm_h5, "w") as hdf_new:
    for uid, emb in hdf.items():
        fasta_id = new_df.loc[new_df["acc_id"] == uid, "fasta_id"].values[0]
        id_tracker[fasta_id] = id_tracker.setdefault(fasta_id, 0) + 1
        fasta_id = f"{fasta_id}_{id_tracker[fasta_id]}"
        new_df.loc[new_df["acc_id"] == uid, "fasta_id"] = fasta_id
        hdf_new.create_dataset(name=fasta_id, data=emb)

col_data = new_df.pop("fasta_id")
new_df.insert(loc=0, column="fasta_id", value=col_data)
new_df.to_csv(csv_, index=False)

## Add french data

In [ ]:
xml_french = Path("raw/3FTs vs receptor dec12.xlsx")

In [3]:
import pandas as pd
import requests
from pyfaidx import Fasta
from pathlib import Path

base = Path(".")
fasta_old_path = base / "raw" / "3and6_new.fasta"  # "3and6_current_alignment.fasta"
csv_old_path = base / "raw" / "Ivan_3FTx.csv"  # "SM_table.csv"
csv_new_path = base / "raw" / "3FTs vs receptor dec12.xlsx"
fasta_new_path = base / "raw" / "tmp.fasta"

fasta_final = base / "3FTx.fasta"
csv_final = base / "3FTx.csv"

## Read in new data (from french guys)

In [4]:
df = pd.read_excel(csv_new_path)
df = df[df["Groups"].notna()]
df = df.drop(columns="SwissProt #")
df = df.reset_index(drop=True)
df.loc[df["Representative"].isna(), "Representative"] = False
df.loc[df["Representative"] == 1.0, "Representative"] = True
df["Representative"] = df["Representative"].astype(bool)
df_new = df.copy()
df_new.head(2)

/usr/local/anaconda3/envs/protspace3d/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,Uniprot #,Entry,Name,Venom,Receptor,Subtype specificity,Activity,Alternative names,Number of CC bonds,PDB code,Referent,Groups,Representative
0,P0C1Y9,3SE1_DENAN,Fasciculin 1,Dendroaspis angusticeps,Acetylcholinesterase (AChE),NaN,Allosteric inhibitor,"Fas1, Fasciculin-I , Fas-I,",4,1FAS,Pascale,1.0,False
1,P0C1Z0,3SE2_DENAN,Fasciculin 2,Dendroaspis angusticeps,Acetylcholinesterase (AChE),NaN,Allosteric inhibitor,"Fas2, Fasciculin-II , Fas-II, toxin F-VII",4,1FSC,Pascale,1.0,True


### Get sequences with UniProt Identifier

In [5]:
# entry names with no UniprotID
na_row_names = df_new.loc[df_new["Uniprot #"].isna(), "Name"].values
print(na_row_names)

# drop rows with NA UniprotID
df_new = df_new.loc[df_new["Uniprot #"].notna()]

['Neurotoxin II' 'Muscarinic toxin 5' 'FS2']


In [6]:
# List of Uniprot identifiers
uniprot_ids = df_new["Uniprot #"].values

# Set the URL for the request
url = (
    "https://rest.uniprot.org/uniprotkb/stream?compressed=false&format=fasta&query="
    + "%20OR%20".join(uniprot_ids)
)

# Send the request to Uniprot
response = requests.get(url).text

In [7]:
# save response to FASTA file
seq, header = "", ""
flag = False
with open(fasta_new_path, "w") as out:
    for line in response.split("\n"):
        line = line.strip()
        if line.startswith(">"):
            if seq and flag:
                out.write(f"{seq}\n")
            header = line.split("|")[1]
            if header in uniprot_ids:
                flag = True
                out.write(f">{header}\n")
                seq = ""
            else:
                flag = False
        elif flag:
            seq += line
    if flag:
        out.write(f"{seq}\n")

## Read-in and parse old data

- remove `>` from Alphabet column
- remove entries having `snake genomics` in column `Major group`
- remove columns with predictions
- extract Uniprot ID (what is possible)
- add protein sequence to CSV file
  - sequences "AAAAAAAAKU" and "AAAAAAAAKT" have no entries in the CSV file?


<!-- - extract sews Figure 1 (3C): https://click.endnote.com/viewer?doi=10.1096%2Ffba.1027&token=WzEzMzUzNzAsIjEwLjEwOTYvZmJhLjEwMjciXQ.P2pjnkkZxmPJrOo-8_f_xWlyKvk
(Long-chain) -->

### BLAST in UniProt - results 100% match

TODO:
- [ ] from all three FASTA files extract all headers, sequences, and taxas
  - [ ] check for duplicated headers
  - [ ] check for duplicated sequences
- [ ] Get all UniProt IDs
  - [ ] extract UniProt IDs from header
  - [ ] BLAST UniProt for 100% sequence using sequence and taxa
    - [ ] Write soffisticated Python scripts
  - [ ] store all found IDs imediatly to a file, thereby they dio not get searched again if rerun
- [ ] Specify data_origin
- [ ] Specify gene_name/prot_name (3FTx / Ly6)
- [ ] Final CSV file contains: acc_id, data_origin, taxa, taxa_id, seq, signal_seq, active_seq, prot_name

### Quick and dirty CSV merge

In [46]:
df = df_old.copy()
df = df.drop(columns=["Alphabeticode"])
df = df.reset_index(drop=True)

df1 = df_new.copy()
df1 = df1.rename(columns={"Uniprot #": "Name in fasta"})
df1["Major group"] = "3FTx_new"
df1 = df1.reset_index(drop=True)

In [47]:
colname_id = "Name in fasta"
# df = pd.merge(left=df, right=df1, on=colname_id, how="outer")
df = pd.concat([df, df1])
colidx = df.pop(colname_id)
df.insert(0, colname_id, colidx)
df.to_csv(csv_final, index=False)
df.head(2)

,Name in fasta,Original fasta header,Major group,Evolutionary order,Species,Major taxon,Entry,Name,Venom,Receptor,Subtype specificity,Activity,Alternative names,Number of CC bonds,PDB code,Referent,Groups,Representative
0,Cbivi_3FTx_000,Cbivi_3FTx_000,3FTx,Basal,Calliophis bivirgatus,Elapidae,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Erythrolamprus_poecilogyrus_A7X3M9,Erythrolamprus_poecilogyrus_A7X3M9,3FTx,Basal,Erythrolamprus poecilogyrus,Colubridae,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Clean-up CSV
- get uniprot ID
  - last element of `Name in fasta` column -> or use `Original fasta header`?
  - get uniprot ID if it starts with G...
  - if it doesn't start with unigeneXXX
  - if something like: `353sp|C0HK` -> 10 char long extract identifier from `Original fasta header` (second entry when split with `|`)
  - if it is at least 5 charcters long
  - check if it can be found in uniprot

In [27]:
df = df_old.copy()
df = df.drop(columns=["Alphabeticode"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 991 entries, 0 to 990
Data columns (total 32 columns):
 #   Column                                      Non-Null Count  Dtype 
---  ------                                      --------------  ----- 
 0   TM prediction                               991 non-null    object
 1   Cell localisation prediction                991 non-null    object
 2   Original fasta header                       991 non-null    object
 3   Name in fasta                               991 non-null    object
 4   Name (existing or suggested)                991 non-null    object
 5   Major group                                 991 non-null    object
 6   Genomic toxins                              27 non-null     object
 7   Preliminary cysteine group                  991 non-null    object
 8   Evolutionary order                          991 non-null    object
 9   Novel function                              65 non-null     object
 10  Species                   

## Report
- French guys dset (`3FTs vs receptor dec12.xlsx`):
  - Three entry names had no UniProtID and were removed: 'Neurotoxin II', 'Muscarinic toxin 5', 'FS2'
- Ivan + MH dset:
  - remove dashes `-` in seqs
  - extract UniProt identifier where possible
  - same ID, different sequence?
    ```
    >Walterinnesia_aegyptia_C0HKZ8
    MECYKCGVSGCHLKITCSAEEKFCYKWQDKISNERWLGCAKTCTEEDTWRVYNSCCTTNLCNI
    >353sp|C0HK
    >sp|C0HKZ8|3NOJ_WALAE Actiflagelin OS=Walterinnesia aegyptia OX=64182 PE=1 SV=2
    MECYKCGVSGCHLKITCSAEEKFCYKWRDKISNERWLGCAKTCTEENTWRVYNSCCTTNLCNP
    ```
    - according to UniProt history one is the old sequence version


  - Given that the **Original fasta header** column in the `SM_table.csv` file had unique names the FASTA file (`3and6_current_alignment.fasta`) was updated with those
  - `3FTs vs receptor dec12.xlsx`
- Clean up CSV
  - Name in fasta
    - What is 3FTx_000, 3FTx_05, ...

In [72]:
seq1 = "MECYKCGVSGCHLKITCSAEEKFCYKWQDKISNERWLGCAKTCTEEDTWRVYNSCCTTNLCNI"
seq2 = "MECYKCGVSGCHLKITCSAEEKFCYKWRDKISNERWLGCAKTCTEENTWRVYNSCCTTNLCNP"
for idx, (s1, s2) in enumerate(zip(seq1, seq2), 1):
    if s1 != s2:
        print(idx, s1, s2)

28 Q R
47 D N
63 I P
